# How to normalize contracted Gaussian-type orbitals (CGTOs)

## Used symbols

Cartesian coordinates $x, y, z$, real numbers.

GTO exponents $\alpha, \alpha_i \alpha_j$, positive real numbers.

Angular momenta $l, m, n$ and their sum $L$, integers $\geq 0$.


## Introduction
This notebook tries to answer several questions (in order).

1. How to determine the normalization factor $N_\text{PGTO}$ for a primitive GTO, so its self-overlap is 1.
2. What is the overlap of two primitive GTOs on the same center, with equal angular momenta $l, m, n$ but different exponents
$\alpha_i$ and $\alpha_j$?
3. How to determine the normalization factor $N_\text{CGTO}$ for a contracted GTO, formed from multiple **normalized** PGTOs.
4. How to compute a sensible contraction factor.

## Adapted from https://aoterodelaroza.github.io/devnotes/critic2-molwfns/

In [1]:
from sympy import *
import numpy as np
from scipy.special import factorial2 as spf2

In [2]:
x, y, z = symbols("x, y, z", real=True)
alpha, alphai, alphaj = symbols("alpha alpha_i alpha_j", real=True, positive=True)
l, m, n, L = symbols("l m n L", integer=True, positive=True)

## Primitive GTO
Centered at $\mathbf{A}$.

$$
G(l, m, n, \mathbf{A}) = (x-A_x)^l (y-A_y)^m (z-A_z)^n \exp^{-\alpha ((x-A_x)^2 + (y-A_y)^2 + (z-A_z)^2)}
$$

and the corresponding self-overlap integral

$$
S = \int_{-\infty}^\infty
 \int_{-\infty}^\infty
   \int_{-\infty}^\infty
     G(l, m, n, \mathbf{A})^2 \text{d}x \text{d}y \text{d}z
$$

The integral can be separated in 3 1d slices along the Cartesian directions.

$$
S = I_x I_y I_z
$$

with $I_x$

$$
I_x = \int_{-\infty}^\infty
 (x-A_x)^{2l} \exp^{-2\alpha(x-A_x)^2} \text{d}x
$$
Expressions for $I_x$ etc. are calculated below.

## 1d self overlap of a primitive GTO

Centered at the origin so $\mathbf{A} = \mathbf{0}$ and same exponent, same l, m and n.

In [3]:
ovlp_expr = x**(2*l) * exp(-2*alpha*x**2)
ovlp_expr

x**(2*l)*exp(-2*alpha*x**2)

In [4]:
ovlp1d = integrate(ovlp_expr, (x, -oo, oo))
ovlp1d

(2*alpha)**(1/2 - l)*gamma(l + 1/2)/(2*alpha)

## Expansion of $\Gamma(n + \frac{1}{2})$

If $n$ is a positive integer  $\Gamma(n + \frac{1}{2})$ can be expanded to 

 $\Gamma(n + \frac{1}{2}) = \sqrt{\pi} ~ (2n - 1)!! ~ 2^{-n}$
 
 with $!!$ being the double factorial.

In [5]:
def expand_gamma(expr, am):
    return expr.subs(gamma(am+Rational(1/2)), sqrt(pi)*factorial2(2*am-1)/2**am)

expand_gamma(ovlp1d, l)

sqrt(pi)*(2*alpha)**(1/2 - l)*factorial2(2*l - 1)/(2*2**l*alpha)

## 3d self overlap of a primitive GTO

$$S = I_x I_y I_z$$

In [6]:
ovlp3d = ovlp1d * ovlp1d.subs(l, m) * ovlp1d.subs(l, n)
ovlp3d

(2*alpha)**(1/2 - l)*(2*alpha)**(1/2 - m)*(2*alpha)**(1/2 - n)*gamma(l + 1/2)*gamma(m + 1/2)*gamma(n + 1/2)/(8*alpha**3)

In [7]:
ovlp3dsimple = simplify(ovlp3d)
ovlp3dsimple

(1/(2*alpha))**(l + m + n + 3/2)*gamma(l + 1/2)*gamma(m + 1/2)*gamma(n + 1/2)

## With expanded Gamma

In [8]:
for am in (l, m, n):
    ovlp3dsimple = expand_gamma(ovlp3dsimple, am)
ovlp3dsimple

pi**(3/2)*(1/(2*alpha))**(l + m + n + 3/2)*factorial2(2*l - 1)*factorial2(2*m - 1)*factorial2(2*n - 1)/(2**l*2**m*2**n)

## Normalization constant for a primitive GTO, so its self overlap is 1

In [15]:
Npgto = 1 / sqrt(ovlp3dsimple)
Npgto

2**(l/2)*2**(m/2)*2**(n/2)*(1/(2*alpha))**(-l/2 - m/2 - n/2 - 3/4)/(pi**(3/4)*sqrt(factorial2(2*l - 1))*sqrt(factorial2(2*m - 1))*sqrt(factorial2(2*n - 1)))

## Verify, that the self overlap of a normalized primitive GTO is actually 1.

In [16]:
ovlp_expr3d = Npgto**2 * ovlp_expr * ovlp_expr.subs(x, y).subs(l, m) * ovlp_expr.subs(x, z).subs(l, n)
ovlp_expr3d

2**l*2**m*2**n*x**(2*l)*y**(2*m)*z**(2*n)*(1/(2*alpha))**(-l - m - n - 3/2)*exp(-2*alpha*x**2)*exp(-2*alpha*y**2)*exp(-2*alpha*z**2)/(pi**(3/2)*factorial2(2*l - 1)*factorial2(2*m - 1)*factorial2(2*n - 1))

In [17]:
simplify(integrate(ovlp_expr3d, (x, -oo, oo), (y, -oo, oo), (z, -oo, oo)))

1

## Overlap of two primitive GTOs

Same center (at the origin), same $l, m$ and $n$, but different exponents $\alpha_i$ and $\alpha_j$.

In [21]:
def get_pgto(exponent):
    """Returns normalized primitive GTO with given exponent."""
    return Npgto.subs(alpha, exponent) * x**l * y**m * z**n * exp(-exponent * (x**2 + y**2 + z**2))

# Two primitive GTOs with different exponents
gtoi = get_pgto(alphai)
gtoj = get_pgto(alphaj)
gtoi

2**(l/2)*2**(m/2)*2**(n/2)*x**l*y**m*z**n*(1/(2*alpha_i))**(-l/2 - m/2 - n/2 - 3/4)*exp(-alpha_i*(x**2 + y**2 + z**2))/(pi**(3/4)*sqrt(factorial2(2*l - 1))*sqrt(factorial2(2*m - 1))*sqrt(factorial2(2*n - 1)))

In [22]:
Sgto = integrate(gtoi*gtoj, (x, -oo, oo), (y, -oo, oo), (z, -oo, oo))
Sgto

2*2**(2*l + 2*m + 2*n + 1/2)*alpha_i**(-l/2 - m/2 + n/2 + 3/4)*alpha_j**(l/2 + m/2 + n/2 + 3/4)*(1 + alpha_j/alpha_i)**(-l - 1/2)*(1 + alpha_j/alpha_i)**(-m - 1/2)*(alpha_i + alpha_j)**(1/2 - n)*gamma(l + 1/2)*gamma(m + 1/2)*gamma(n + 1/2)/(pi**(3/2)*alpha_i*(alpha_i + alpha_j)*factorial2(2*l - 1)*factorial2(2*m - 1)*factorial2(2*n - 1))

In [23]:
simplify(Sgto)

(2/(alpha_i + alpha_j))**(l + m + n + 3/2)*(alpha_i*alpha_j)**(l/2 + m/2 + n/2 + 3/4)

Slightly simplified with $L = l + m + n$:

$$S = 2^{L+\frac{3}{2}} \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}$$

from this a normalization constant for a contracted GTO (CGTO) $N_\text{CGTO}$ can be determined. See below.

## Self-overlap of a contracted GTO

Same center, same $l, m$ and $n$, different contraction coefficients $d_i$ and $d_j$, as well as different exponents $\alpha_i$ and $\alpha_j$.

$$S_\text{CGTO} = N_\text{CGTO}^2 2^{L + \frac{3}{2}} \sum_{i, j} d_i d_j \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} = 1$$

yields for $N_\text{CGTO}$

$$N_\text{CGTO} =  \left[  2^{L + \frac{3}{2}} \sum_{i, j} d_i d_j \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} \right]^{-\frac{1}{2}}$$

Alternatively, one can insert the expression for normalization constant of a PGTO (see *Npgto* above).

$$N_\text{PGTO} = \frac{2^{L+\frac{3}{4}} \alpha^{\frac{L}{2}+\frac{3}{4}}}{\pi^\frac{3}{4}
 \sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}$$
 
The expression can be rearranged to

$$\alpha^{\frac{L}{2}+\frac{3}{4}} = \frac{N_\text{PGTO} \pi^\frac{3}{4}
 \sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}{2^{L+\frac{3}{4}} }$$
 
which can be inserted into $N_\text{CGTO}$:

$$N_\text{CGTO} =  \left[
 2^{L + \frac{3}{2}} \sum_{i, j} \frac{d_i d_j}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} \alpha_i^{\frac{L}{2} + \frac{3}{4}} \alpha_j^{\frac{L}{2} + \frac{3}{4}} \right]^{-\frac{1}{2}}$$
 
$$N_\text{CGTO} =  \left[
 2^{L + \frac{3}{2}} \sum_{i, j} \frac{d_i d_j}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 %alpha_i
 \frac{N_{\text{PGTO},i} \pi^\frac{3}{4}
 \sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}{2^{L+\frac{3}{4}} }
 %alpha_j
 \frac{N_{\text{PGTO},j} \pi^\frac{3}{4}
 \sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}{2^{L+\frac{3}{4}} }
 \right]^{-\frac{1}{2}}
$$

$$N_\text{CGTO} =  \left[
 2^{L + \frac{3}{2}} (2l-1)!!(2m-1)!!(2n-1)!!
 % sum start
 \sum_{i, j} \frac{d_i d_j}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 %alpha_i
 \frac{N_{\text{PGTO},i} \pi^\frac{3}{4}}{2^{L+\frac{3}{4}} }
 %alpha_j
 \frac{N_{\text{PGTO},j} \pi^\frac{3}{4}}{2^{L+\frac{3}{4}} }
 \right]^{-\frac{1}{2}}
$$

$$N_\text{CGTO} =  \left[
 2^{L + \frac{3}{2}} (2l-1)!!(2m-1)!!(2n-1)!! \pi^\frac{3}{2}
 % sum start
 \sum_{i, j} \frac{d_i d_j}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 %alpha_i
 \frac{N_{\text{PGTO},i}}{2^{L+\frac{3}{4}} }
 %alpha_j
 \frac{N_{\text{PGTO},j}}{2^{L+\frac{3}{4}} }
 \right]^{-\frac{1}{2}}
$$

$$N_\text{CGTO} =  \left[
 \frac{2^{L + \frac{3}{2}}}{2^{2L+\frac{3}{2}}} (2l-1)!!(2m-1)!!(2n-1)!! \pi^\frac{3}{2}
 % sum start
 \sum_{i, j} \frac{d_i d_j}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 %alpha_i
 N_{\text{PGTO},i}
 %alpha_j
 N_{\text{PGTO},j}
 \right]^{-\frac{1}{2}}
$$

$$N_\text{CGTO} =  \left[
 \frac{(2l-1)!!(2m-1)!!(2n-1)!! \pi^\frac{3}{2}}{2^L}
 % sum start
 \sum_{i, j} \frac{d_i d_j N_{\text{PGTO},i} N_{\text{PGTO},j}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 \right]^{-\frac{1}{2}}
$$

$$N_\text{CGTO} =  \left[
(2l-1)!!(2m-1)!!(2n-1)!!  \frac{\pi^\frac{3}{2}}{2^L}
 % sum start
 \sum_{i, j} \frac{d_i d_j N_{\text{PGTO},i} N_{\text{PGTO},j}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}}
 \right]^{-\frac{1}{2}}
$$

But this leads nowhere ;)

In practice one wants to split the normalization constant in two parts: one part only dependent on all exponents, contraction coefficients and the total angular momentum of a shell $L = l + m + n$ and a part, directly dependent on $l, m$ and $n$.

The final contraction coefficient is obtained as:

$$
d_k = d_{k,0} N_\text{CGTO} N_\text{PGTO}
$$

with $d_{k,0}$ being the initial contraction coefficient, as obtained, e.g., from the basissetexchange or a file storing basis set information, $N_\text{CGTO}$ is the normalization constant for a contracted GTO (dependent only on $L$ and $\alpha$ and $d$) and
$N_\text{PGTO}$ the normalization constant for a primitive GTO, dependent on $l, m$ and $n$.

$$
d_k =
d_{k,0}
%CGTO
\underbrace{\left[  2^{L + \frac{3}{2}} \sum_{i, j} d_i d_j \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} \right]^{-\frac{1}{2}}
\frac{2^{L+\frac{3}{4}} \alpha_k^{\frac{L}{2}+\frac{3}{4}}}{\pi^\frac{3}{4}}}_{\text{depending on} ~ \alpha, d, L}
\underbrace{\frac{1}{\sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}}_{\text{depending on} ~ l, m, n}
$$

$$
d_k =
d_{k,0}
%CGTO
\underbrace{\left[  \pi^\frac{3}{2} 2^{L + \frac{3}{2}} \sum_{i, j} d_i d_j \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} \right]^{-\frac{1}{2}}
2^{L+\frac{3}{4}} \alpha_k^{\frac{L}{2}+\frac{3}{4}}}_{\text{depending on} ~ \alpha, d, L}
\underbrace{\frac{1}{\sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}}_{\text{depending on} ~ l, m, n}
$$

$$
d_k =
d_{k,0}
%CGTO
\underbrace{\left[  \frac{\pi^\frac{3}{2}}{2^L} \sum_{i, j} d_i d_j \frac{(\alpha_i \alpha_j)^{\frac{L}{2} + \frac{3}{4}}}{(\alpha_i + \alpha_j)^{L+\frac{3}{2}}} \right]^{-\frac{1}{2}}
\alpha_k^{\frac{L}{2}+\frac{3}{4}}}_{\text{depending on} ~ \alpha, d, L}
\underbrace{\frac{1}{\sqrt{(2l-1)!!(2m-1)!!(2n-1)!!}}}_{\text{depending on} ~ l, m, n}
$$

The $lmn$ part must be taken into account inside the actual integral code, the other part could be precomputed outside of the actual integral code.